# Bayesian Spatio-Temporal Graph Transformer Network (B-STAR) for Multi-Aircraft Trajectory Prediction
Author: Yutian Pang, Arizona State University


Email: yutian.pang@asu.edu

# Part 1: IFF ASDE-X Flight Track Data Processing with PySpark and Hadoop
This is a demonstration of using PySpark and Hadoop for large-scale processing of IFF ASDE-X data. In practice, this data processing would be performed on a server or high-performance cluster via ssh.

## Module Requirements

This Jupyter notebook has been tested with:
- Ubuntu 20.04 LTS (and 18.04 LTS)
- Python 3.8.5 (and 3.8.10)
- Spark 3.1.1 with Hadoop3.2 (and Spark 3.2.1 with Hadoop3.2)

The software in parenthesis were tested together. Other combinations of Ubuntu, Python, and Spark should be verified for compatibility. See [this](https://stackoverflow.com/questions/58700384/how-to-fix-typeerror-an-integer-is-required-got-type-bytes-error-when-tryin) article for further guidance.

### Instructions for Windows 10 Users

1. **Install Ubuntu on Windows 10 with Windows Subsystem for LInux (WSL)**
    - Windows 10 users with admin privileges can enable Windows Subsystem for Linux (WSL) following [these](https://docs.microsoft.com/en-us/windows/wsl/install-win10) directions.
        
        
2. **Install Anaconda in the Ubuntu terminal**
    - A user can then install Anaconda on their WSL Ubuntu distribution following [these](https://gist.github.com/kauffmanes/5e74916617f9993bc3479f401dfec7da) instructions. 
        
        
3. **Download and unzip Spark on WSL**
    - Identify the distribution of Spark and Hadoop you require [here](https://spark.apache.org/downloads.html). 
    - In your Ubuntu terminal window execute the ```wget``` command followed by the download link in your chosen download directory (likely the ```HOME``` directory). 
    - Then, unzip the downloaded .tgz file with ```tar -xvzf [fname]```.



## Installing the required Python packages
The required Python packages for this module are:
- **[```pyspark```]**(http://spark.apache.org/docs/latest/api/python/getting_started/index.html)
    - This is the Python API for Apache Spark. We will be using the distributed processing features and backend SQL queries for structured data.
- **[```apache-sedona```]**(https://sedona.apache.org/)
    - Formerly Geospark, Apache Sedona extends the Resilient Distributed Dataset (RDD), the core data structure in Apache Spark, to accommodate big geospatial data in a cluster environment.
    
In the Ubuntu or Anaconda terminal, execute ```pip install pyspark apache-sedona```. This will install both the ```pyspark``` and ```apache-sedona``` packages. 

## Setting Environment Variables
The Spark codes (note: Improve this description) retrieve the ```SPARK_HOME```, ```PYTHONPATH```, ```PYSPARK_PYTHON```, and ```PYSPARK_DRIVER_PYTHON``` system variables. Either (Option 1) these are set in the shell environment in the ```.bash_profile``` script or (Option 2) in the Python script prior to calling the ```pyspark``` module.
- ### Option 1: Add environment variables to the ```.bash_profile``` script

    Open the ```.bash_profile``` script in your text editor. On Ubuntu systems, this script is usually found in your ```HOME``` directory ```~/```. If this file does not yet exist (or is empty) you can create one. Then add the following ```export``` statements for each variable you want to add and add them to the path. For example:

    ```export SPARK_HOME="$HOME/spark-X.X.X-bin-hadoopX.X"```

    ```export PYTHONPATH="$HOME/anacond3/bin/python3.8"```

    ```export PYSPARK_PYTHON="$HOME/anacond3/bin/python3.8"```

    ```export PYSPARK_DRIVER_PYTHON="$HOME/anacond3/bin/python3.8"```

    ```export PATH="$SPARK_HOME/bin:$PATH"```

- ### Option 2: Add the environment variables in the Python script using the ```os``` package

    ```import os```
       
    ```os.environ["SPARK_HOME"] = '~/spark-3.1.1-bin-hadoop3.2'```

    ```os.environ["PYTHONPATH"] = '~/anaconda3/bin/python3.8'```

    ```os.environ['PYSPARK_PYTHON'] = '~/anaconda3/bin/python3.8'```

    ```os.environ['PYSPARK_DRIVER_PYTHON'] = '~/anaconda3/bin/python3.8'```


## Step 1: Define SQL Schema for IFF ASDE-X Data with ```pyspark``` Structures

In [ ]:
# Retrieve several different SQL types from pyspark
from pyspark.sql.types import (ShortType, StringType, StructType,StructField,LongType, IntegerType, DoubleType)

#Create load_schema function which returns variable 'myschema' specifically for recTye=3.
def load_schema():
    myschema = StructType([
        StructField("recType", ShortType(), True),  # 1  //track point record type number
        StructField("recTime", StringType(), True),  # 2  //seconds since midnigght 1/1/70 UTC
        StructField("fltKey", LongType(), True),  # 3  //flight key
        StructField("bcnCode", IntegerType(), True),  # 4  //digit range from 0 to 7
        StructField("cid", IntegerType(), True),  # 5  //computer flight id
        StructField("Source", StringType(), True),  # 6  //source of the record
        StructField("msgType", StringType(), True),  # 7
        StructField("acId", StringType(), True),  # 8  //call sign
        StructField("recTypeCat", IntegerType(), True),  # 9
        StructField("lat", DoubleType(), True),  # 10
        StructField("lon", DoubleType(), True),  # 11
        StructField("alt", DoubleType(), True),  # 12  //in 100s of feet
        StructField("significance", ShortType(), True),  # 13 //digit range from 1 to 10
        StructField("latAcc", DoubleType(), True),  # 14
        StructField("lonAcc", DoubleType(), True),  # 15
        StructField("altAcc", DoubleType(), True),  # 16
        StructField("groundSpeed", IntegerType(), True),  # 17 //in knots
        StructField("course", DoubleType(), True),  # 18  //in degrees from true north
        StructField("rateOfClimb", DoubleType(), True),  # 19  //in feet per minute
        StructField("altQualifier", StringType(), True),  # 20  //Altitude qualifier (the “B4 character”)
        StructField("altIndicator", StringType(), True),  # 21  //Altitude indicator (the “C4 character”)
        StructField("trackPtStatus", StringType(), True),  # 22  //Track point status (e.g., ‘C’ for coast)
        StructField("leaderDir", IntegerType(), True),  # 23  //int 0-8 representing the direction of the leader line
        StructField("scratchPad", StringType(), True),  # 24
        StructField("msawInhibitInd", ShortType(), True),  # 25 // MSAW Inhibit Indicator (0=not inhibited, 1=inhibited)
        StructField("assignedAltString", StringType(), True),  # 26
        StructField("controllingFac", StringType(), True),  # 27
        StructField("controllingSec", StringType(), True),  # 28
        StructField("receivingFac", StringType(), True),  # 29
        StructField("receivingSec", StringType(), True),  # 30
        StructField("activeContr", IntegerType(), True),  # 31  // the active control number
        StructField("primaryContr", IntegerType(), True),
        # 32  //The primary(previous, controlling, or possible next)controller number
        StructField("kybrdSubset", StringType(), True),  # 33  //identifies a subset of controller keyboards
        StructField("kybrdSymbol", StringType(), True),  # 34  //identifies a keyboard within the keyboard subsets
        StructField("adsCode", IntegerType(), True),  # 35  //arrival departure status code
        StructField("opsType", StringType(), True),  # 36  //Operations type (O/E/A/D/I/U)from ARTS and ARTS 3A data
        StructField("airportCode", StringType(), True),  # 37
        StructField("trackNumber", IntegerType(), True),  # 38
        StructField("tptReturnType", StringType(), True),  # 39
        StructField("modeSCode", StringType(), True)  # 40
    ])
    return myschema

## Step 2: Use Sedona to register Spark Objects

In [ ]:
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

spark = SparkSession.\
        builder.\
        master("local[*]").\
        appName("Sector_IFF_Parser").\
        config("spark.serializer", KryoSerializer.getName).\
        config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
        config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.1.1-incubating,'
           'org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
        getOrCreate()

SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext

# Load Data
## We use ASDE-X flight recordings here 

In [ ]:
# The path to the demonstration IFF ASDE-X data
from paraatm.io.iff import read_iff_file

iff_fpath = "../../miscellaneous/gnats-fpgen/IFF_SFO_ASDEX_ABC456.csv"

#Current paraatm function
%timeit df_patm = read_iff_file(iff_fpath)
df_patm.head()

In [ ]:
iff_schema = load_schema()
def load_data(fname,verbose=False):
    df = spark.read.csv(fname, header=False, sep=",", schema=iff_schema)
    if verbose:
        print("Number of Lines: {}".format(df.count()))
    return df

In [ ]:
# load data into Sedona
%timeit df = load_data(data_path)

In [ ]:
# In this work, we are only interested in the flight ID, timestamps, and coordinates(latitude, longitude, altitude)
cols = ['recType', 'recTime', 'acId', 'lat', 'lon', 'alt']
df = df.select(*cols).filter(df['recType']==3).withColumn("recTime", df['recTime'].cast(IntegerType()))
df.show(5)

### Build Spatial Dataframe with Sedona

In [ ]:
# time window to query
duration = 4 # hours # para-atm time window filtering function
t_start = 1564668000 + (date-20190801)*24*3600 # Aug 1st, 2pm, 2019, UTC
t_end = t_start + 3600*duration

In [ ]:
# register pyspark df in SQL
df.registerTempTable("pointtable")  # now we have a registered SQL table running backened in the system

# create shape column in geospark
spatialdf = spark.sql(
  """
  SELECT ST_Point(CAST(lat AS Decimal(24, 20)), CAST(lon AS Decimal(24, 20))) AS geom, recTime, acId, alt
  FROM pointtable
  WHERE recTime>={} AND recTime<={}
  """.format(t_start, t_end))

spatialdf.createOrReplaceTempView("spatialdf")
spatialdf.show(5, truncate=False)

In [ ]:
# Count the number of points after the first query
spatialdf.count()

### Range Rectangular Query around KATL

In [ ]:
katl = [33.6366996, -84.4278640, 10.06]  # https://www.airnav.com/airport/katl
r = 0.2 # rectangular query range
vs = 0.3 # vertical threshold unit: x100 feet

In [ ]:
# ST_PolygonFromEnvelope (MinX:decimal, MinY:decimal, MaxX:decimal, MaxY:decimal, UUID1, UUID2, ...)
range_query_result = spark.sql(
  """
    SELECT DISTINCT acId
    FROM spatialdf
    WHERE ST_Contains(ST_PolygonFromEnvelope({}, {}, {}, {}), geom) AND alt>{}
  """.format(katl[0]-r, katl[1]-r, katl[0]+r, katl[1]+r, katl[2]+vs))

In [ ]:
# Number of flight IDs after the second query
range_query_result.count()

In [ ]:
range_query_result.show(5)

In [ ]:
df_result = spark.sql(
  """
    SELECT *
    FROM spatialdf
    WHERE ST_Contains(ST_PolygonFromEnvelope({}, {}, {}, {}), geom) AND alt>{}
  """.format(katl[0]-r, katl[1]-r, katl[0]+r, katl[1]+r, katl[2]))

# Count the number of points after the second query
df_result.count()

# Data anonymization

Anonymization is a type of data sanitization technique to remove identifiable information from the data. In this work, we perform two operations to anonymize the data while retaining useful geo-spatial features.

- Normalize the timestamp by the earliest time in the current dataframe.
- Mask the real flight IDs into integers

In [ ]:
# create relevant timestamp column
df_result.createOrReplaceTempView("spatialdf")
df = spark.sql(
"""
    SELECT acId, recTime-{} AS t, geom, alt
    FROM spatialdf
""".format(t_start)
)
df.show(5, False)

In [ ]:
# change acId Into integers
# have to use pyspark ml features
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="acId", outputCol="FacId")
df_new = indexer.fit(df).transform(df).drop('acId')
df_new.show(20, False)

In [ ]:
df_new.createOrReplaceTempView("spatialdf")

df = spark.sql(
"""
    SELECT t, CAST(FacId AS Integer), ST_X(geom) as lat, ST_Y(geom) as lon, alt
    FROM spatialdf
"""
)

## Show the data after anonymization

In [ ]:
df.show(5, False)

# Some small modifications to the data

### Modification 1: 
#### Resample the time series with an interval $dt$

In [ ]:
dt = 10
t_start = 0
t_end = 3600 * duration

t_interval = list(range(t_start, t_end, dt))
df = df[df.t.isin(t_interval)]

In [ ]:
df.show(15)

### Modification 2:
Change the origin of the coordinate system to the airport center
- void in real case

In [ ]:
df.createOrReplaceTempView("spatialdf")

df2 = spark.sql(
"""
    SELECT t, FacId, lat-{} AS Lat, lon-{} AS Lon, alt
    FROM spatialdf
""".format(katl[0], katl[1])
)

### Save into csv

In [ ]:
# Make sure the data types are correct
df.toPandas()
df.dtypes

In [ ]:
# save data with altitude
csv_name = 'KATL_r_{}_date_{}_range_{}_wAltitude.csv'.format(r, date, duration)
df.toPandas().T.to_csv(csv_name, sep=',', index=False, header=False)
# df.coalesce(1).write.csv(csv_name, sep=',')

# save data without altitude dimension
csv_name = 'KATL_r_{}_date_{}_range_{}.csv'.format(r, date, duration)
df.drop('alt').toPandas().T.to_csv(csv_name, sep=',', index=False, header=False)
# df.coalesce(1).write.csv(csv_name, sep=',')